In [1]:
import nltk
import pandas as pd
nltk.download('reuters')
nltk.download('punkt')

# Extract fileids from the reuters corpus
fileids = reuters.fileids()

# Initialize empty lists to store categories and raw text
categories = []
text = []

# Loop through each file id and collect each files categories and raw text
for file in fileids:
    categories.append(reuters.categories(file))
    text.append(reuters.raw(file))

# Combine lists into pandas dataframe. reutersDf is the final dataframe. 
reutersDf = pd.DataFrame({'ids':fileids, 'categories':categories, 'text':text})

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Florence\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Florence\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NameError: name 'reuters' is not defined